In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.storage import LocalFileStore
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings


cache_dir = LocalFileStore("./.cache/")

llm = ChatOpenAI(temperature=0.1)

splitter = CharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
    separator="\n",
)

loader = UnstructuredFileLoader("files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

Created a chunk of size 717, which is longer than the specified 600
Created a chunk of size 608, which is longer than the specified 600
Created a chunk of size 642, which is longer than the specified 600
Created a chunk of size 1444, which is longer than the specified 600
Created a chunk of size 1251, which is longer than the specified 600
Created a chunk of size 1012, which is longer than the specified 600
Created a chunk of size 1493, which is longer than the specified 600
Created a chunk of size 819, which is longer than the specified 600
Created a chunk of size 1458, which is longer than the specified 600
Created a chunk of size 1411, which is longer than the specified 600
Created a chunk of size 742, which is longer than the specified 600
Created a chunk of size 669, which is longer than the specified 600
Created a chunk of size 906, which is longer than the specified 600
Created a chunk of size 703, which is longer than the specified 600
Created a chunk of size 1137, which is lon

In [2]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriever = vectorstore.as_retriever()

# vectorstore.similarity_search("Aaronson은 유죄인가요?")


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up,
            --------
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def map_docs(inputs):
    documents = inputs["context"]
    return "\n\n".join([doc.page_content for doc in documents])


memory = ConversationBufferMemory(memory_key="history", return_messages=True)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


stuff_chain = {"context": retriever} | RunnableLambda(map_docs)

chain = (
    {"context": stuff_chain, "question": RunnablePassthrough()}
    | RunnablePassthrough.assign(history=load_memory)
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

In [3]:
invoke_chain("Is Aaronson guilty?")

content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [4]:
invoke_chain("What message did he write in the table?")

content='He wrote: "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE" on the table.'


In [5]:
invoke_chain("Who is Julia?")

content='Julia is a character who was once in a relationship with the protagonist.'


In [6]:
invoke_chain("What is her relationship with him?")

content='Julia had a past relationship with the protagonist, but their current interaction seems to be filled with contempt and dislike.'
